<a href="https://colab.research.google.com/github/sean-otoole/HODD/blob/main/parkinsons_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

tsv_path = '/content/drive/MyDrive/HODD/Kamath_et_al_2022/Homo_features.tsv'
df = pd.read_csv(tsv_path, sep='\t')
print(df.head())

print(df.shape)

    RP11-34P13.3 RP11-34P13.3.1  Gene Expression
0        FAM138A        FAM138A  Gene Expression
1          OR4F5          OR4F5  Gene Expression
2   RP11-34P13.7   RP11-34P13.7  Gene Expression
3   RP11-34P13.8   RP11-34P13.8  Gene Expression
4  RP11-34P13.14  RP11-34P13.14  Gene Expression
(41624, 3)
